In [1]:
#for Preview version in BAM

import dotenv
import os

from genai.credentials import Credentials
from genai.schemas import GenerateParams
from genai.model import Model

dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=1000,
    min_new_tokens=1,
)
model = Model(model="meta-llama/llama-2-70b-chat",credentials=creds, params=params)
# model = Model(model="codellama/codellama-34b-instruct",credentials=creds, params=params)

In [2]:
def build_prompt(longinput):
    prompt_template = """[INST]be a business analyst, summary the requirement provided.\n
        <<SYS>>
        requirement:```{requirement}```
        <</SYS>>
        [/INST]
        summary:"""

    i = 0
    prompts = []
    while i+4000 <= len(longinput):
        chunk = [longinput[i:i+4000]]
        print(f"input>>{chunk}")
        prompts += [prompt_template.format(requirement=chunk)]
        i+=4000
    return prompts

In [3]:
from docx import Document

def build_summary(input_file_path):

    adoc = Document(input_file_path)

    summaries = []
    prompts = []

    longinput = ""

    for paragraph in adoc.paragraphs:
        inputtext = paragraph.text
        longinput += inputtext

    prompts += build_prompt(longinput)
        

    for table in adoc.tables:
        longinput = ""
        for row in table.rows:
            for cell in row.cells:
                inputtext = cell.text
                longinput += inputtext

        prompts += build_prompt(longinput)

    for response in model.generate(prompts):
        summaries += [response.generated_text]
        print(f"summary>>{response.generated_text}")

    return summaries


In [4]:
from pathlib import Path
import os

for path in Path('requirement example').rglob('*.docx'):
    print(path.name)
    inputfile = os.path.join(Path('requirement example'),path.name)
    summaries = build_summary(inputfile)

# output_file_path = input_file_path.replace(
#         '.docx', '-out.docx')

# adoc.save(output_file_path)

MEC - Developer Requirements and LBGUPS.docx
input>>['MEC Discovery and Beta Customer PilotPhase 1 - RequirementsAugust  - December 2021Discovery (Phase 1) Scope / OfferingOur phase 1 will start with offering a BETA offering on TELUS MEC. This beta offering will be tested using early adopter “friendly” Enterprise customers. This initial service will consider the foundational building blocks of our TELUS MEC network, and offer a basic subset of developer features to cater towards use cases which are 10ms or above within the range of our 4 sites. Developers will have partial self service for set up, configuration, lifecycle management, and monitoring of their applications using our TMEC service within Canada only. User / Enterprise Developer FunctionsPartial management from TELUS (creation / provisioning), with partial self service capabilities for user on lifecycle / container management and monitoring / alertingTELUS will manually provision container for customer based on initial consu